# Importando Pacotes

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importando Dados Consolidados

In [16]:
indicadores = pd.read_excel("../wrangled_data/indicadores.xlsx", index_col = "Data")
btc = pd.read_excel("../wrangled_data/serie_bitcoin.xlsx", index_col = "Data")

In [17]:
btc.head()

,Fechamento,Máxima,Mínima,Abertura
Data,,,,
2010-07-17,0.049510,0.04951,0.04951,0.04951
2010-07-18,0.085840,0.08585,0.05941,0.05941
2010-07-19,0.080800,0.09307,0.07855,0.09090
2010-07-20,0.074733,0.08181,0.07426,0.08181
2010-07-21,0.079210,0.07921,0.06634,0.07425


# Gerenciamento de Risco

#### Stop Fixo por operação

In [18]:
stop_fixo = pd.DataFrame()
stop_fixo["Retorno"] = btc["Fechamento"].pct_change()
stop_fixo["Alvo_Contínuo"] = pd.Series(stop_fixo["Retorno"])
stop_fixo["Alvo_Binário"] = np.where(stop_fixo["Retorno"] > 0, 1, 0)

STOP = 0.4
stop_fixo["Stop_Long"] = np.where(btc["Abertura"]/btc["Mínima"] > (1+STOP), 1, 0)
stop_fixo["Stop_Short"] = np.where(btc["Máxima"]/btc["Abertura"] > (1+STOP), 1, 0)
stop_fixo[["Alvo_Contínuo", "Alvo_Binário", "Stop_Long", "Stop_Short"]] = stop_fixo[["Alvo_Contínuo", "Alvo_Binário", "Stop_Long", "Stop_Short"]].shift(-1) 
stop_fixo.dropna(inplace = True)

stop_fixo.head()

,Retorno,Alvo_Contínuo,Alvo_Binário,Stop_Long,Stop_Short
Data,,,,,
2010-07-18,0.733791,-0.058714,0.0,0.0,0.0
2010-07-19,-0.058714,-0.075083,0.0,0.0,0.0
2010-07-20,-0.075083,0.059902,1.0,0.0,0.0
2010-07-21,0.059902,-0.293713,0.0,1.0,0.0
2010-07-22,-0.293713,0.113296,1.0,0.0,0.0


#### Stop Ajustado pela Volatilidade

In [19]:
stop_vol = pd.DataFrame()
stop_vol["Retorno"] = btc["Fechamento"].pct_change()
stop_vol["Alvo_Contínuo"] = pd.Series(stop_vol["Retorno"]).shift(-1)
stop_vol["Alvo_Binário"] = np.where(stop_vol["Retorno"] > 0, 1, 0)

# Distâncias entre a abertura e os extremos do candle
stop_vol["d_max"] = (btc["Máxima"]/btc["Abertura"])-1
stop_vol["d_min"] = (btc["Abertura"]/btc["Mínima"])-1

# Desvio-padrão amostral das distâncias da série 
sd_d_max, sd_d_min = stop_vol["d_max"].std(), stop_vol["d_min"].std()

stop_vol["stop_short"] = np.where(stop_vol["d_max"] > 2*sd_d_max, 1, 0)
stop_vol["stop_short"] = stop_vol["stop_short"].shift()

stop_vol["stop_long"] = np.where(stop_vol["d_min"] > 2*sd_d_min, 1, 0)
stop_vol["stop_long"] = stop_vol["stop_long"].shift()
stop_vol.dropna(inplace = True)

stop_vol.head()

,Retorno,Alvo_Contínuo,Alvo_Binário,d_max,d_min,stop_short,stop_long
Data,,,,,,,
2010-07-18,0.733791,-0.058714,1,0.445043,0.000000,0.0,0.0
2010-07-19,-0.058714,-0.075083,0,0.023872,0.157225,1.0,0.0
2010-07-20,-0.075083,0.059902,0,0.000000,0.101670,0.0,0.0
2010-07-21,0.059902,-0.293713,1,0.066801,0.119234,0.0,0.0
2010-07-22,-0.293713,0.113296,0,0.020073,0.454462,0.0,0.0


# Indicadores Técnicos

In [20]:
inds_tec = pd.DataFrame()
inds_tec["Retorno"] = btc["Fechamento"].pct_change()

# Desvios-Padrão Móveis
inds_tec["STD5"] = inds_tec["Retorno"].rolling(5).std()
inds_tec["STD10"] = inds_tec["Retorno"].rolling(10).std()
inds_tec["STD15"] = inds_tec["Retorno"].rolling(15).std()

inds_tec["MM10"] = btc["Fechamento"].rolling(10).mean()

# Proporções de Corpo dos Candles
inds_tec["PROP"] = (btc["Fechamento"]-btc["Abertura"])/(btc["Máxima"]-btc["Mínima"])

# Direções dos Dias Anteriores
inds_tec["D0"] = np.where(btc["Fechamento"] > btc["Abertura"], 1, 0)
inds_tec["D1"] = inds_tec["D0"].shift(1)
inds_tec["D2"] = inds_tec["D0"].shift(2)
inds_tec["D3"] = inds_tec["D0"].shift(3)

# Z-Score Móvel
tmp = pd.DataFrame()
tmp["MM30"] = btc["Fechamento"].rolling(30).mean()
tmp["MM200"] = btc["Fechamento"].rolling(200).mean()
tmp["STD200"] = btc["Fechamento"].rolling(200).std()
inds_tec["Z30"] = (tmp["MM30"]-tmp["MM200"])/tmp["STD200"]

# RSL
inds_tec["RSL"] = (btc["Fechamento"]/inds_tec["MM10"])-1

# Tendência
tmp = pd.DataFrame()
tmp["MM10"] = inds_tec["MM10"]
tmp["MM100"] = btc["Fechamento"].rolling(100).mean()
tmp["MM200"] = btc["Fechamento"].rolling(200).mean()
tmp.dropna(inplace = True)
tmp["TEND_ALTA"] = np.where((tmp["MM10"] > tmp["MM100"]) & (tmp["MM100"] > tmp["MM200"]), 1, 0)
tmp["TEND_BAIXA"] = np.where((tmp["MM10"] < tmp["MM100"]) & (tmp["MM100"] < tmp["MM200"]), 1, 0)
tmp.drop(["MM10", "MM100", "MM200"], axis = 1, inplace = True)
inds_tec = inds_tec.join(tmp)

df = indicadores.join(inds_tec)
df.drop(["Retorno", "MM10"], axis = 1, inplace = True)
inds_tec.tail()

,Retorno,STD5,STD10,STD15,MM10,PROP,D0,D1,D2,D3,Z30,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,
2022-08-26,-0.063454,0.029734,0.037968,0.031944,21626.399184,-0.857026,0,1.0,0.0,1.0,-0.987760,-0.064421,0.0,1.0
2022-08-27,-0.008592,0.029712,0.037911,0.030915,21300.441696,-0.448967,0,0.0,1.0,0.0,-0.988751,-0.058265,0.0,1.0
2022-08-28,-0.020422,0.027687,0.037784,0.030737,20943.680485,-0.801606,0,0.0,0.0,1.0,-0.990861,-0.061783,0.0,1.0
2022-08-29,0.032325,0.035852,0.026318,0.032935,20883.905483,0.902405,1,0.0,0.0,0.0,-0.990882,-0.028683,0.0,1.0
2022-08-30,-0.000862,0.034708,0.025712,0.033036,20793.739530,-0.209487,0,1.0,0.0,0.0,-0.989776,-0.025312,0.0,1.0


# Transformação das Variáveis

#### Variações Percentuais
* SPX (já transformado)
* DXY (já transformado)
* GLD (já transformado)
* 10YIELD (já transformado)
* HASH_RATE30
* AN_INFLATION

In [21]:
df["HASH_RATE30"] = df["HASH_RATE30"].pct_change()
df["AN_INFLATION"] = df["AN_INFLATION"].pct_change()
df.head()

,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,THERMOCAP,...,STD15,PROP,D0,D1,D2,D3,Z30,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,,,,,,,,
2009-01-03,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-04,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Suavização com médias móveis

* FEE_MEAN: Variação % da MM 7d
* TRENDS: Média Movel 7d
* NUPL: Média Movel 7d

In [22]:
tmp = pd.DataFrame()
tmp["MM7"] = df["FEE_MEAN"].rolling(7).mean()
df["FEE_MEAN"] = tmp["MM7"].pct_change()

In [23]:
df["TRENDS"] = df["TRENDS"].rolling(7).mean()

In [24]:
df["NUPL"] = df["NUPL"].rolling(7).mean()

#### Z-Score Móvel
* END_ATIVOS: (Med Movel 30d - Med Movel 200d)/Media Movel do Desvio Padrão Med Mov 200d
* CDD

In [25]:
tmp = pd.DataFrame()
tmp["MM30"] = df["END_ATIVOS"].rolling(30).mean()
tmp["MM200"] = df["END_ATIVOS"].rolling(200).mean()
tmp["STD200"] = df["END_ATIVOS"].rolling(200).std()
df["END_ATIVOS"] = (tmp["MM30"] - tmp["MM200"])/tmp["STD200"]

In [26]:
tmp = pd.DataFrame()
tmp["MM30"] = df["CDD"].rolling(9).mean()
tmp["MM200"] = df["CDD"].rolling(200).mean()
tmp["STD200"] = df["CDD"].rolling(200).std()
df["CDD"] = (tmp["MM30"] - tmp["MM200"])/tmp["STD200"]

# Exportação das bases

In [27]:
variaveis = df.dropna()
variaveis.to_excel("../wrangled_data/variaveis.xlsx", index_label = "Data")
stop_fixo.to_excel("../wrangled_data/stop_fixo.xlsx", index_label = "Data")
stop_vol.to_excel("../wrangled_data/stop_vol.xlsx", index_label = "Data")
inds_tec.to_excel("../wrangled_data/inds_tec.xlsx", index_label = "Data")

variaveis.head()

,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,THERMOCAP,...,STD15,PROP,D0,D1,D2,D3,Z30,RSL,TEND_ALTA,TEND_BAIXA
Data,,,,,,,,,,,,,,,,,,,,,
2011-02-01,-0.036790,1.346414,0.012031,0.392474,3.962088,4.259223,4.472763,5.023446,5.562989,0.000001,...,0.106261,0.578913,1.0,1.0,1.0,0.0,1.644952,0.496305,1.0,0.0
2011-02-02,-0.038023,1.361285,0.010910,0.238807,3.934975,3.738760,4.252111,4.955148,5.412498,0.000001,...,0.097873,0.085702,1.0,1.0,1.0,1.0,1.660428,0.431013,1.0,0.0
2011-02-03,0.011130,1.342658,0.013371,0.104872,3.764052,3.166427,3.801977,4.686634,5.053846,0.000001,...,0.099331,-0.295465,0.0,1.0,1.0,1.0,1.677207,0.318935,1.0,0.0
2011-02-04,0.210286,1.355250,0.018983,0.168706,4.298699,3.752805,4.627094,5.360298,5.914965,0.000002,...,0.090169,0.508096,1.0,0.0,1.0,1.0,1.691116,0.427581,1.0,0.0
2011-02-07,0.144713,1.340026,0.017436,0.149473,4.426032,3.034375,4.279002,5.252936,5.725800,0.000002,...,0.093420,-0.161603,0.0,0.0,1.0,1.0,1.730407,0.254542,1.0,0.0


In [28]:
variaveis.describe().T

,count,mean,std,min,25%,50%,75%,max
AN_INFLATION,2713.0,4.755483e-03,1.245778e-01,-4.877244e-01,-8.056348e-02,-3.015408e-04,0.083772,0.786302
END_ATIVOS,2713.0,5.871610e-01,7.797118e-01,-1.638629e+00,7.113734e-02,6.954409e-01,1.117788,2.154090
HASH_RATE30,2713.0,5.202947e-03,9.306300e-03,-2.381645e-02,-1.327391e-04,3.690872e-03,0.008521,0.062593
FEE_MEAN,2713.0,1.049520e-01,5.197359e+00,-7.002315e-01,-2.191210e-02,1.124371e-03,0.025580,270.680756
MMULT,2713.0,1.437808e+00,1.028708e+00,2.258761e-01,9.001055e-01,1.171377e+00,1.603321,13.296906
MRGO140,2713.0,9.098588e-01,2.742954e+00,-7.301030e+00,-5.386948e-01,3.675058e-01,1.835431,16.228291
MRGO28,2713.0,5.742224e-01,2.832907e+00,-1.020606e+01,-6.272124e-01,1.321123e-01,1.141071,22.476845
MVRV,2713.0,1.845122e+00,9.209548e-01,3.868285e-01,1.250241e+00,1.677026e+00,2.180964,7.387878
MVRV_Z,2713.0,1.684979e+00,1.928493e+00,-8.300573e-01,4.300454e-01,1.210987e+00,2.221339,12.547296
THERMOCAP,2713.0,8.186225e-07,7.342204e-07,1.202472e-07,3.259260e-07,5.742111e-07,0.000001,0.000006
